# [모듈 2.1] 모델 훈련 스텝 및 모델 등록 스텝 개발 (SageMaker Model Building Pipeline 훈련 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 0. 모델 훈련 및 모델 등록 개요 
- 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 2. 모델 훈련 코드 확인
- 3. 모델 훈련 스텝 개발 및 실행
    - 아래의 3단계를 진행하여 SageMaker Model Building Pipeline 에서 훈련 스텝 개발 함. 아래의 (1), (2) 단계는 옵션이지만, 실제 현업 개발시에 필요한 단계이기에 실행을 권장 드립니다.
        - (1) **[로컬 노트북 인스턴스]**에서 다커 컨테이너로 훈련 코드 실행 (로컬 모드로 불리움)
        - (2) 세이지메이커 호스트 모드(로컬 다커 컨테이너 사용) 및 실험(Experiment)사용하여 훈련 코드 실행
        - (3) [필수] SageMaker Model Building Pipeline 에서 모델 훈련 스텝 및 모델 등록 스텝  실행
    

---
### 노트북 커널
- 이 워크샵은 노트북 커널이 `conda_python3` 를 사용합니다. 다른 커널일 경우 변경 해주세요.
---



# 0. 모델 훈련 및 모델 등록 개요
- 모델 훈련에 대한 부분은 scratch/4.1.Train-Pipeline.ipynb 노트북을 참조 바랍니다.
- 모델 등록 스텝은 여기를 참조 바랍니다. --> [모델 레지스트리로 모델 등록 및 배포](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry.html)

# 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 이전 단계(전처리)에서 결과 파일을 로딩 합니다. 실제 훈련에 제공되는 데이터를 확인하기 위함 입니다.
---

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
sm_client = boto3.client("sagemaker")

%store -r 
# 노트북에 저장되어 있는 변수를 보기 위해서는 주석을 제거하고 실행하시면 됩니다.
# %store  

In [2]:
! aws s3 ls {train_preproc_data_uri} --recursive

2021-08-03 12:39:19     682602 sagemaker-pipeline-step-by-step-phase01/preporc/train.csv


In [3]:
train_prep_df = pd.read_csv(train_preproc_data_uri)
train_prep_df

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,8913.668763,80513.668763,54,94,0,1,750,3000,99207,...,0,0,1,0,0,0,1,0,1,0
1,0,19746.724395,26146.724395,41,165,0,1,750,2950,95632,...,0,0,0,1,0,0,0,1,0,1
2,0,11652.969918,22052.969918,57,155,0,1,750,3000,93203,...,0,0,1,0,0,0,0,1,0,1
3,0,11260.930936,115960.930936,39,80,0,1,750,3000,85208,...,0,0,1,0,0,0,1,0,1,0
4,0,27987.704652,31387.704652,39,60,0,1,750,3000,91792,...,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,18052.611626,67152.611626,42,103,1,1,750,3000,93654,...,0,0,1,0,0,0,1,0,1,0
3996,0,34949.202468,51749.202468,23,6,0,3,750,3000,94305,...,0,0,0,1,1,0,0,0,1,0
3997,0,4063.701410,9963.701410,44,35,0,2,750,2550,95476,...,0,0,1,0,0,0,0,1,0,1
3998,0,17390.520451,20490.520451,22,38,0,1,750,3000,90680,...,0,1,0,0,0,0,0,1,0,1


### 기본 훈련 변수 및 하이퍼파라미터 설정

기본 하이퍼파라미터 외에 `scale_pos_weight` 는 레이블이 뷸균형이 되어 있을 경우에, 사용할 수 있는 하나의 방법 입니다. 일반적인 경우에 잘 작동합니다.

In [4]:
from sagemaker.xgboost.estimator import XGBoost

bucket = sagemaker_session.default_bucket()
prefix = project_prefix

estimator_output_path = f's3://{bucket}/{prefix}/training_jobs'
train_instance_count = 1

def get_pos_scale_weight(df, label):
    '''
    1, 0 의 레이블 분포를 계산하여 클래스 가중치 리턴
    예: 1: 10, 0: 90 이면 90/10 = 9 를 제공함. 
    호출:
        class_weight = get_pos_scale_weight(train_prep_df, label='fraud')
    '''
    fraud_sum = df[df[label] == 1].shape[0]
    non_fraud_sum = df[df[label] == 0].shape[0]
    class_weight = int(non_fraud_sum / fraud_sum)
    print(f"fraud_sum: {fraud_sum} , non_fraud_sum: {non_fraud_sum}, class_weight: {class_weight}")
    return class_weight
    
class_weight = get_pos_scale_weight(train_prep_df, label='fraud')


hyperparameters = {
       "scale_pos_weight" : class_weight,    
        "max_depth": "3",
        "alpha" : "0.2", 
        "eta": "0.333",
        "min_child_weight": "7",
        "objective": "binary:logistic",
        "num_round": "100",
}



fraud_sum: 131 , non_fraud_sum: 3869, class_weight: 29


# 2. 모델 훈련 코드 확인

훈련 코드는 크게 아래와 같이 구성 되어 있습니다.
- 커맨드 인자로 전달된 변수 내용 확인
- 훈련 데이터를 로딩 합니다.
- xgboost의 cross-validation(cv) 로 훈련 합니다.
- 훈련 및 검증 데이터 세트의 roc-auc 값을 metrics_data 에 저장
    - 모델 훈련시 생성되는 지표(예: loss 등)는 크게 두가지 방식으로 사용할 수 있습니다.
        - 클라우드 워치에서 실시간으로 지표 확인
        - 하이퍼 파라미터 튜닝(HPO) 에서 평가 지표로 사용 (예: validation:roc-auc)
        - 참조 --> [Monitor and Analyze Training Jobs Using Metrics](https://docs.amazonaws.cn/en_us/sagemaker/latest/dg/training-metrics.html)
        - 참조: XGBoost Framework 에는 이미 디폴트로 정의된 metric definition이 있어서, 정의된 규칙에 따라서 모델 훈련시에 print() 를 하게 되면, metric 이 클라우드 워치 혹은 HPO에서 사용이 가능합니다.
        
            
```            
Name				Regex
validation:auc	.*\[[0-9]+\].*#011validation-auc:([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*
train:auc	    .*\[[0-9]+\].*#011train-auc:([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*

실제 코드에 위의 Regex 형태로 print() 반영
print(f"[0]#011train-auc:{train_auc_mean}")
print(f"[1]#011validation-auc:{validation_auc_mean}")
```
    
- 훈련 성능을 나타내는 지표를 저장합니다. (metrics.json)
- 훈련이 모델 아티펙트를 저장 합니다.

---

In [29]:
!pygmentize src/xgboost_script.py

import os
import sys
import pickle
import xgboost as xgb
import argparse
import pandas as pd
import json

import pandas as pd
pd.options.display.max_rows=20
pd.options.display.max_columns=10

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    ###################################
    ## 커맨드 인자 처리
    ###################################    
    
    # Hyperparameters are described here
    parser.add_argument('--scale_pos_weight', type=int, default=50)    
    parser.add_argument('--num_round', type=int, default=100)
    parser.add_argument('--max_depth', type=int, default=3)
    parser.add_argument('--min_child_weight', type=float, default=1.0)    
    parser.add_argument('--alpha', type=float, default=0.0)        
    parser.add_argument('--eta', type=float, default=0.2)
    parser.add_argument('--objective', type=str, default='binary:logistic')
    parser.add_argument('--nfold', type=int, default=5)
    parser.add_argument('--early_stopping_rounds', type=int, defaul

# 3. 모델 훈련 스텝 개발 및 실행
---



## (1) 로컬 노트북 인스턴스에서 로컬 모드(로컬 다커 컨테이너 사용)로 훈련 코드 실행

In [6]:

xgb_estimator_local = XGBoost(
    entry_point = "xgboost_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = 'local',
    framework_version = "1.0-1"
)
    
xgb_estimator_local.fit(inputs = {'train': train_preproc_data_uri},
                      wait=True,                        
                 )


Creating lfptigj4la-algo-1-vjn5q ... 
Creating lfptigj4la-algo-1-vjn5q ... done
Attaching to lfptigj4la-algo-1-vjn5q
lfptigj4la-algo-1-vjn5q | INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
lfptigj4la-algo-1-vjn5q | INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
lfptigj4la-algo-1-vjn5q | INFO:sagemaker_xgboost_container.training:Invoking user training script.
lfptigj4la-algo-1-vjn5q | INFO:sagemaker-containers:Module xgboost_script does not provide a setup.py. 
lfptigj4la-algo-1-vjn5q | Generating setup.py
lfptigj4la-algo-1-vjn5q | INFO:sagemaker-containers:Generating setup.cfg
lfptigj4la-algo-1-vjn5q | INFO:sagemaker-containers:Generating MANIFEST.in
lfptigj4la-algo-1-vjn5q | INFO:sagemaker-containers:Installing module with the following command:
lfptigj4la-algo-1-vjn5q | /miniconda3/bin/python3 -m pip install . 
lfptigj4la-algo-1-vjn5q | Processing /opt/ml/code
lfptigj4la-algo-1-vjn5q | Building wheels for collected package

Failed to delete: /tmp/tmpc_k8pbv0/algo-1-vjn5q Please remove it manually.


===== Job Complete =====


## (2) 세이지메이커 호스트 모드(로컬 다커 컨테이너 사용) 및 실험(Experiment)사용하여 훈련 코드 실행

### 실험(Experiment) 세팅
- Amazon SageMaker 실험은 기계 학습 실험을 구성, 추적, 비교 및 평가할 수 있는 Amazon SageMaker 의 기능입니다
- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 실험을 통한 Machine Learning 관리](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/experiments.html)
- sagemaker experiment는 추가적인 패키지를 설치하여야 합니다. 0.0.Setup-Environment 가 실행이 안되었다고 하면, `!pip install --upgrade sagemaker-experiments` 를 통해 설치 해주세요.
- 여기서는 boto3 API를 통해서 실험을 생성합니다. SageMaker Python SDK를 통해서도 가능합니다.


In [7]:
# !pip install --upgrade sagemaker-experiments

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

from datetime import datetime

sm = boto3.client('sagemaker')


# 설험에 대한 이름을 생성 합니다.
experiment_name = project_prefix + '-single-train'

# 실험이 존재하지 않으면 생성하고, 그렇지 않으면 지나갑니다.
try:
    response = sm_client.describe_experiment(ExperimentName=experiment_name)
    print(f"Experiment:{experiment_name} already exists")    
    
except:
    response = sm_client.create_experiment(
        ExperimentName = experiment_name,
        Description = 'Experiment for fraud detection',
    )
    print(f"Experiment:{experiment_name} is created")        


Experiment:sagemaker-pipeline-step-by-step-phase01-single-train already exists


### 하이퍼 파리미터 변경 실험
- max_depth 5개의 값을 바꾸면서 5개의 훈련잡을 실행합니다.
    - ```for i, max_depth_num in enumerate([1,3,5,7,10]):```
    - 만약에 리소스 제한의 에러가 발생하면, 5개를 2개 정도로 줄여서 실행 해주세요.
- 위에서 생성한 Experiment 안에 5개의 Trial(시도) 를 생성합니다.
- xgb_estimator 에 각각의 하이파라미터를 인자로 제공합니다.
- xgb_estimator.fit()에 Experiment의 설정 파일을 제공합니다.
    - 1개의 실험, 각각의 시도가 설정되어 훈련을 시작 합니다.

In [8]:
instance_type = 'ml.m5.xlarge'
for i, max_depth_num in enumerate([1,3,5,7,10]):
    hyperparameters = {
           "scale_pos_weight" : class_weight,    
            "max_depth": f"{max_depth_num}",
            "alpha" : "0", 
            "eta": "0.3",
            "min_child_weight": "1",
            "objective": "binary:logistic",
            "num_round": "100",
    }
    
    # 시도 이름 생성
    ts = datetime.now().strftime('%Y-%m-%d-%H-%M-%S-%f')
    trial_name = experiment_name + f"-{ts}"

    # 1개의 실험 안에 시도를 생성함.
    response = sm_client.create_trial(
        ExperimentName = experiment_name,
        TrialName = trial_name,
    )    
    
    # 실험 설정: 실험 이름, 시도 이름으로 구성
    experiment_config = {
        'ExperimentName' : experiment_name,
        'TrialName' : trial_name,
        "TrialComponentDisplayName" : 'Training',
    }    

    
    xgb_estimator = XGBoost(
        entry_point = "xgboost_script.py",
        source_dir = "src",
        output_path = estimator_output_path,
        code_location = estimator_output_path,
        hyperparameters = hyperparameters,
        role = role,
        instance_count = train_instance_count,
        instance_type = instance_type,
        framework_version = "1.0-1")

    xgb_estimator.fit(inputs = {'train': train_preproc_data_uri},
                          experiment_config = experiment_config, # 실험 설정 제공
                          wait=False,                        
                     )

    



INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2021-08-03-12-45-07-708
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2021-08-03-12-45-08-165
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2021-08-03-12-45-08-619
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2021-08-

In [9]:
## 마지막 estimator의 로그 출력
xgb_estimator.logs()

2021-08-03 12:45:16 Starting - Starting the training job...
2021-08-03 12:45:17 Starting - Launching requested ML instancesProfilerReport-1627994715: InProgress
...
2021-08-03 12:46:10 Starting - Preparing the instances for training......
2021-08-03 12:47:15 Downloading - Downloading input data...
2021-08-03 12:47:43 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module xgboost_script does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  Created wheel for xgboost-script: filename=xgboost_script-1.0.0-py2.py3-none

###  실험 결과 보기
위의 실험한 결과를 확인 합니다.
- 각각의 훈련잡의 시도에 대한 훈련 사용 데이터, 모델 입력 하이퍼 파라미터, 모델 평가 지표, 모델 아티펙트 결과 위치 등의 확인이 가능합니다.
- **아래의 모든 내용은 SageMaker Studio 를 통해서 직관적으로 확인이 가능합니다.**

In [10]:
from sagemaker.analytics import ExperimentAnalytics
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 5
pd.options.display.max_colwidth = 50

search_expression = {
    "Filters": [
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}


trial_component_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
)

trial_component_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,alpha,eta,max_depth,min_child_weight,num_round,objective,sagemaker_container_log_level,sagemaker_job_name,sagemaker_program,sagemaker_region,sagemaker_submit_directory,scale_pos_weight,train:auc - Min,train:auc - Max,train:auc - Avg,train:auc - StdDev,train:auc - Last,train:auc - Count,validation:auc - Min,validation:auc - Max,validation:auc - Avg,validation:auc - StdDev,validation:auc - Last,validation:auc - Count,train - MediaType,train - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments
0,sagemaker-xgboost-2021-08-03-12-45-15-173-aws-...,Training,arn:aws:sagemaker:ap-northeast-2:057716757052:...,366743142698.dkr.ecr.ap-northeast-2.amazonaws....,1.0,ml.m5.xlarge,30.0,"""0""","""0.3""","""10""","""1""","""100""","""binary:logistic""",20.0,"""sagemaker-xgboost-2021-08-03-12-45-15-173""","""xgboost_script.py""","""ap-northeast-2""","""s3://sagemaker-ap-northeast-2-057716757052/sa...",29.0,0.9443,0.9443,0.9443,0.0,0.9443,1.0,0.7802,0.7802,0.7802,0.0,0.7802,1.0,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,NaN,s3://sagemaker-ap-northeast-2-057716757052/sag...,[sagemaker-pipeline-step-by-step-phase01-singl...,[sagemaker-pipeline-step-by-step-phase01-singl...
1,sagemaker-xgboost-2021-08-03-12-45-12-381-aws-...,Training,arn:aws:sagemaker:ap-northeast-2:057716757052:...,366743142698.dkr.ecr.ap-northeast-2.amazonaws....,1.0,ml.m5.xlarge,30.0,"""0""","""0.3""","""7""","""1""","""100""","""binary:logistic""",20.0,"""sagemaker-xgboost-2021-08-03-12-45-12-381""","""xgboost_script.py""","""ap-northeast-2""","""s3://sagemaker-ap-northeast-2-057716757052/sa...",29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,NaN,NaN,[sagemaker-pipeline-step-by-step-phase01-singl...,[sagemaker-pipeline-step-by-step-phase01-singl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,sagemaker-xgboost-2021-07-23-02-12-54-542-aws-...,Training,arn:aws:sagemaker:ap-northeast-2:057716757052:...,366743142698.dkr.ecr.ap-northeast-2.amazonaws....,1.0,ml.m5.2xlarge,30.0,"""0""","""0.3""","""6""","""1""","""100""","""binary:logistic""",20.0,"""sagemaker-xgboost-2021-07-23-02-12-54-542""","""xgboost_script.py""","""ap-northeast-2""","""s3://sagemaker-ap-northeast-2-057716757052/sa...",29.0,0.9014,0.9014,0.9014,0.0,0.9014,1.0,0.7845,0.7845,0.7845,0.0,0.7845,1.0,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,NaN,s3://sagemaker-ap-northeast-2-057716757052/sag...,[Trial-2021-07-23-01-41-41-524747],[sagemaker-pipeline-step-by-step-phase01-singl...
29,sagemaker-xgboost-2021-07-23-01-43-58-289-aws-...,Training,arn:aws:sagemaker:ap-northeast-2:057716757052:...,366743142698.dkr.ecr.ap-northeast-2.amazonaws....,1.0,ml.m5.xlarge,30.0,"""0""","""0.3""","""6""","""1""","""100""","""binary:logistic""",20.0,"""sagemaker-xgboost-2021-07-23-01-43-58-289""","""xgboost_script.py""","""ap-northeast-2""","""s3://sagemaker-ap-northeast-2-057716757052/sa...",29.0,0.9014,0.9014,0.9014,0.0,0.9014,1.0,0.7845,0.7845,0.7845,0.0,0.7845,1.0,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,NaN,s3://sagemaker-ap-northeast-2-057716757052/sag...,[Trial-2021-07-23-01-41-41-524747],[sagemaker-pipeline-step-by-step-phase01-singl...


### 모델 평가 지표에 순서에 따른 시도 보기
- 아래는 모델 평가 지표에 따른 순서로 보여주기 입니다.

In [11]:

trial_component_training_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
    sort_by="metrics.validation:auc.max",        
    sort_order="Descending",
    metric_names=["validation:auc"],
    parameter_names=["hidden_channels", "epochs", "dropout", "optimizer"],
)

trial_component_training_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,validation:auc - Min,validation:auc - Max,validation:auc - Avg,validation:auc - StdDev,validation:auc - Last,validation:auc - Count,train - MediaType,train - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments
0,sagemaker-xgboost-2021-08-03-12-45-07-708-aws-...,Training,arn:aws:sagemaker:ap-northeast-2:057716757052:...,0.8257,0.8257,0.8257,0.0,0.8257,1.0,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,NaN,s3://sagemaker-ap-northeast-2-057716757052/sag...,[sagemaker-pipeline-step-by-step-phase01-singl...,[sagemaker-pipeline-step-by-step-phase01-singl...
1,sagemaker-xgboost-2021-08-03-12-45-08-165-aws-...,Training,arn:aws:sagemaker:ap-northeast-2:057716757052:...,0.8225,0.8225,0.8225,0.0,0.8225,1.0,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,NaN,s3://sagemaker-ap-northeast-2-057716757052/sag...,[sagemaker-pipeline-step-by-step-phase01-singl...,[sagemaker-pipeline-step-by-step-phase01-singl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,sagemaker-xgboost-2021-08-03-12-45-15-173-aws-...,Training,arn:aws:sagemaker:ap-northeast-2:057716757052:...,0.7802,0.7802,0.7802,0.0,0.7802,1.0,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,NaN,s3://sagemaker-ap-northeast-2-057716757052/sag...,[sagemaker-pipeline-step-by-step-phase01-singl...,[sagemaker-pipeline-step-by-step-phase01-singl...
29,sagemaker-xgboost-2021-08-03-12-45-12-381-aws-...,Training,arn:aws:sagemaker:ap-northeast-2:057716757052:...,NaN,NaN,NaN,NaN,NaN,NaN,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,None,s3://sagemaker-ap-northeast-2-057716757052/sag...,NaN,NaN,[sagemaker-pipeline-step-by-step-phase01-singl...,[sagemaker-pipeline-step-by-step-phase01-singl...


## (3) SageMaker Pipeline에서  실행 
- 모델 훈련 스텝과 모델 등록 스텝 두가지를 실행합니다.
    - 두 개의 단계가 서로 의존성이 있기에, 두 개의 단계를 연결을 합니다.

---



### 모델 빌딩 파이프라인 변수 생성



In [12]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)


training_instance_count = ParameterInteger(
    name="TrainInstanceCount",
    default_value=1
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)


input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)


### 모델 학습을 위한 학습단계 정의 

본 단계에서는 SageMaker의 [XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) 알고리즘을 이용하여 학습을 진행할 것입니다. XGBoost 알고리즘을 이용하도록 Estimator를 구성합니다. 보편적인 학습스크립트를 이용하여 입력 채널에서 정의한 학습데이터를 로드하고, 하이퍼파라미터 설정을 통해 학습을 설정하고, 모델을 학습한 후 `model_dir`경로에 학습된 모델을 저장합니다. 저장된 모델은 이후 호스팅을 위해 사용됩니다. 

학습된 모델이 추출되어 저장될 경로 또한 명시되었습니다. 

`training_instance_type`파라미터가 사용된 것을 확인합니다. 이 값은 본 예제의 파이프라인에서 여러번 사용됩니다. 본 단계에서는 estimator를 선언할 때 전달되었습니다. 


In [13]:
hyperparameters = {
       "scale_pos_weight" : class_weight,    
        "max_depth": "3",
        "alpha" : "0", 
        "eta": "0.3",
        "min_child_weight": "1",
        "objective": "binary:logistic",
        "num_round": "100",
}


xgb_train = XGBoost(
    entry_point = "xgboost_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = training_instance_type,
    framework_version = "1.0-1")

이전 단계에서 (프로세싱) 전처리 훈련, 검증 데이터 세트를 입력으로 제공 합니다.

In [14]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="FraudScratchTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data= train_preproc_data_uri,
            content_type="text/csv"
        ),
    },
)

### 모델 등록 스텝
- 모델들이 저장이 될 그룹 이름을 제공하고, 모델 등록 스텝을 정의 합니다.
- 모델 등록 단계의 개발자 가이드 
    - [모델 등록기 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-register-model)
    - [모델 레지스트리로 모델 등록 및 배포](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry.html)
- 모델 그룹 리스팅 API:  [ListModelPackageGroups](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_ListModelPackageGroups.html)   

In [15]:
model_package_group_name = f"{project_prefix}"
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Sample model package group"
}

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

from sagemaker.workflow.step_collections import RegisterModel

step_register = RegisterModel(
    name= f"{project_prefix}-XgboostRegisterModel",
    estimator=xgb_train,
    image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,    
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    #model_metrics=model_metrics,
)

### 모델 빌딩 파이프라인 정의
- 파이프라인과 실험(Experiment)가 통합이 되었습니다. 이를 위한 실험 설정 파일을 같이 제공합니다.

In [16]:
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig


pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        training_instance_type,        
        input_data,
        model_approval_status,
    ],
    pipeline_experiment_config=PipelineExperimentConfig(
      ExecutionVariables.PIPELINE_NAME,
      ExecutionVariables.PIPELINE_EXECUTION_ID
    ),    
    steps=[step_train, step_register],
)

In [17]:
import json

definition = json.loads(pipeline.definition())
# definition

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [18]:
pipeline.upsert(role_arn=role)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase01',
 'ResponseMetadata': {'RequestId': '6fd3a6dc-8e87-4659-b3e4-a649c6cb4529',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6fd3a6dc-8e87-4659-b3e4-a649c6cb4529',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '112',
   'date': 'Tue, 03 Aug 2021 12:48:28 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [19]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [20]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase01',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase01/execution/qnfh4tj6sa8f',
 'PipelineExecutionDisplayName': 'execution-1627994909134',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 8, 3, 12, 48, 29, 67000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 8, 3, 12, 48, 29, 67000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '40768c5f-8693-4f82-aac0-1aff91cadca5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '40768c5f-8693-4f82-aac0-1aff91cadca5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '453',
   'date': 'Tue, 03 Aug 2021 12:48:28 GMT'},
  'RetryAttempts': 0}}

In [21]:

execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [22]:
execution.list_steps()

[{'StepName': 'sagemaker-pipeline-step-by-step-phase01-XgboostRegisterModel',
  'StartTime': datetime.datetime(2021, 8, 3, 12, 52, 28, 6000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 3, 12, 52, 28, 759000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model-package/sagemaker-pipeline-step-by-step-phase01/6'}}},
 {'StepName': 'FraudScratchTrain',
  'StartTime': datetime.datetime(2021, 8, 3, 12, 48, 30, 529000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 3, 12, 52, 27, 409000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:training-job/pipelines-qnfh4tj6sa8f-fraudscratchtrain-hardrupxva'}}}]

## 모델 레지스트리에서 모델 등록 확인
위에서 등록한 모델 그룹 이름을 통해서 어떤 모델이 등록되었는지를 확인 합니다.
- 등록된 모델 버전에 대한 보기 --> [모델 버전의 세부 정보 보기](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry-details.html)

In [23]:
# 위에서 생성한 model_package_group_name 을 인자로 제공 합니다.
response = sm_client.list_model_packages(ModelPackageGroupName= model_package_group_name)
response

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'sagemaker-pipeline-step-by-step-phase01',
   'ModelPackageVersion': 6,
   'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model-package/sagemaker-pipeline-step-by-step-phase01/6',
   'CreationTime': datetime.datetime(2021, 8, 3, 12, 52, 28, 645000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'},
  {'ModelPackageGroupName': 'sagemaker-pipeline-step-by-step-phase01',
   'ModelPackageVersion': 5,
   'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model-package/sagemaker-pipeline-step-by-step-phase01/5',
   'CreationTime': datetime.datetime(2021, 7, 25, 10, 39, 29, 449000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'},
  {'ModelPackageGroupName': 'sagemaker-pipeline-step-by-step-phase01',
   'ModelPackageVersion': 4,
   'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:057716

#### 등록된 모델 버전의 상세 정보 확인

In [24]:
ModelPackageArn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
sm_client.describe_model_package(ModelPackageName=ModelPackageArn)

{'ModelPackageGroupName': 'sagemaker-pipeline-step-by-step-phase01',
 'ModelPackageVersion': 6,
 'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model-package/sagemaker-pipeline-step-by-step-phase01/6',
 'CreationTime': datetime.datetime(2021, 8, 3, 12, 52, 28, 645000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3',
    'ImageDigest': 'sha256:04889b02181f14632e19ef6c2a7d74bfe699ff4c7f44669a78834bc90b77fe5a',
    'ModelDataUrl': 's3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase01/training_jobs/pipelines-qnfh4tj6sa8f-FraudScratchTrain-HardRUpxva/output/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m5.xlarge'],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.t2.medium', 'ml.m5.xlarge'],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPa

### 아티펙트 경로 추출
위의 훈련 스텝이 완료되면 실행해주세요

In [25]:
from IPython.display import display as dp
def get_train_artifact(execution, client, job_type,  kind=0):
    '''
    kind: 0 --> train
    kind: 2 --> test
    '''
    response = execution.list_steps()
    dp("response: ", response)
    proc_arn = response[1]['Metadata'][job_type]['Arn'] # 1은 훈련 스텝
    train_job_name = proc_arn.split('/')[-1]
    # print("train_job_name: ", train_job_name)
    response = client.describe_training_job(TrainingJobName = train_job_name)
    # print("\nresponse: ", response)    
    train_model_artifact = response['ModelArtifacts']['S3ModelArtifacts']    
    
    return train_model_artifact

import boto3
client = boto3.client("sagemaker")
    
train_model_artifact = get_train_artifact(execution, client,job_type='TrainingJob', kind=0)
print(" train_model_artifact: ", train_model_artifact)


'response: '

[{'StepName': 'sagemaker-pipeline-step-by-step-phase01-XgboostRegisterModel',
  'StartTime': datetime.datetime(2021, 8, 3, 12, 52, 28, 6000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 3, 12, 52, 28, 759000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model-package/sagemaker-pipeline-step-by-step-phase01/6'}}},
 {'StepName': 'FraudScratchTrain',
  'StartTime': datetime.datetime(2021, 8, 3, 12, 48, 30, 529000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 3, 12, 52, 27, 409000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:training-job/pipelines-qnfh4tj6sa8f-fraudscratchtrain-hardrupxva'}}}]

 train_model_artifact:  s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase01/training_jobs/pipelines-qnfh4tj6sa8f-FraudScratchTrain-HardRUpxva/output/model.tar.gz


In [26]:
image_uri = xgb_train.image_uri

훈련 모델 아티펙트와, 훈련시 사용한 다커 이미지의 경로를 저장 합니다.

In [27]:
%store train_model_artifact
%store image_uri

Stored 'train_model_artifact' (str)
Stored 'image_uri' (str)
